### Streaming Data 의 Window Operations
- 특정 시간 동안의 데이터를 모아서 처리
    - 최근 30분 동안 가장 많이 발생한 단어
    - 최근 1시간 동안 가장 많이 검색된 단어(실검 순위)
    - 지난 1시간 동안 거래가 가장 많았던 제품, 고객 등등
    - windowLenght, slideInterval: streaming Context 정의 때 설정한 Seconds()의 배수로 설정

In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
import pyspark

In [4]:
from pyspark import SparkConf, SparkContext

In [5]:
from pyspark.streaming import StreamingContext

In [6]:
sc = pyspark.SparkContext(master="local[2]")

In [7]:
ssc = StreamingContext(sc, 10)

In [8]:
ssc.checkpoint("C:/spark/data/checkpoint")

In [10]:
lines = ssc.socketTextStream("localhost",9999)

In [11]:
import re

In [13]:
words =  lines.flatMap(lambda line: re.split("\W+", line))

In [14]:
words = words.filter(lambda word: len(word)>0)

In [15]:
wordsLower = words.map(lambda w: w.lower())

In [16]:
wordAndOne = wordsLower.map(lambda w: (w, 1))

In [17]:
wordCount = wordAndOne.reduceByKeyAndWindow(lambda v1, v2: v1 + v2, lambda v1, v2: v1 - v2,windowDuration=30,slideDuration=10)

In [18]:
sortedWC = wordCount.transform(lambda r:r.sortBy(lambda x: x[1], False))

In [19]:
sortedWC.pprint()

In [20]:
ssc.start()

-------------------------------------------
Time: 2021-08-21 20:42:50
-------------------------------------------

-------------------------------------------
Time: 2021-08-21 20:43:00
-------------------------------------------

-------------------------------------------
Time: 2021-08-21 20:43:10
-------------------------------------------

-------------------------------------------
Time: 2021-08-21 20:43:20
-------------------------------------------

-------------------------------------------
Time: 2021-08-21 20:43:30
-------------------------------------------

-------------------------------------------
Time: 2021-08-21 20:43:40
-------------------------------------------

-------------------------------------------
Time: 2021-08-21 20:43:50
-------------------------------------------

-------------------------------------------
Time: 2021-08-21 20:44:00
-------------------------------------------



In [21]:
ssc.awaitTermination

<bound method StreamingContext.awaitTermination of <pyspark.streaming.context.StreamingContext object at 0x000001C5E6FCC2E0>>

-------------------------------------------
Time: 2021-08-21 20:44:10
-------------------------------------------

-------------------------------------------
Time: 2021-08-21 20:44:20
-------------------------------------------

-------------------------------------------
Time: 2021-08-21 20:44:30
-------------------------------------------

-------------------------------------------
Time: 2021-08-21 20:44:40
-------------------------------------------
('you', 2)
('are', 1)
('hey', 1)
('what', 1)
('doing', 1)

-------------------------------------------
Time: 2021-08-21 20:44:50
-------------------------------------------
('you', 2)
('are', 1)
('boring', 1)
('hey', 1)
('what', 1)
('doing', 1)
('so', 1)
('today', 1)

-------------------------------------------
Time: 2021-08-21 20:45:00
-------------------------------------------
('you', 2)
('are', 1)
('boring', 1)
('hey', 1)
('what', 1)
('doing', 1)
('so', 1)
('today', 1)

-------------------------------------------
Time: 2021-08-21 2